In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
climate_type_dict = {'1':'BSk',
 '9': 'Dfb',
 '4': 'Cfa',
 '7': 'Csb',
 '8': 'Dfa',
 '3': 'BWk',
 '10': 'Dfc',
 '2': 'BWh',
 '6': 'Csa',
 '11': 'Dsb',
 '0': 'BSh',
 '5': 'Cfb',
 '12': 'Dsc',
 '13': 'Dwa',
 '14': 'Dwb'}

In [4]:
all_cols = add_train.columns.tolist()

In [5]:
target=["contest-tmp2m-14d__tmp2m"]

In [6]:
train_df =  pd.read_csv('fe_v1_train.csv')
test_df  =  pd.read_csv('fe_v1_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

climateregions__climateregion


In [7]:
exclude_cols = ['index', 'startdate']
best_cols = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(best_cols))

Main features: 63


In [8]:
for c in all_cols:
    if 'contest-precip-14d' in c:
        print(c)

contest-precip-14d__precip


In [9]:
add_train['month'].unique()

array([ 9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8])

In [10]:
def calculate_season(month):
    if month == 12 or month == 1 or month == 2:
        return 4
    if month == 9 or month == 10 or month == 11:
        return 3
    if month == 7 or month == 8 or month == 6:
        return 2
    if month == 3 or month == 4 or month == 5:
        return 1

In [11]:
train_df['month'] = add_train['month']
test_df['month'] = add_test['month']

In [12]:
train_df['season'] = train_df['month'].apply(calculate_season)
test_df['season'] = test_df['month'].apply(calculate_season)

In [13]:
train_df['loc_group'] = add_train['loc_group']
test_df['loc_group'] = add_test['loc_group']

In [14]:
train_df['year'] = add_train['year']
test_df['year'] = add_test['year']

In [15]:
def map_climate_type(x):
    return climate_type_dict[str(x)]

In [16]:
train_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)
test_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)

In [17]:
precip_col = 'contest-precip-14d__precip'
wind_col = 'contest-wind-h500-14d__wind-hgt-500'

In [18]:
group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']

In [19]:
check_cols = [c for c in train_df.columns if c not in target and c not in exclude_cols]

In [20]:
df = pd.concat([train_df[check_cols], test_df[check_cols]], axis=0)
df.shape

(407088, 68)

In [21]:
for c in check_cols:
    if df[c].isnull().sum() > 0:
        print(c)

In [22]:
min_precip_df_by_month_df = df.groupby(group_cols)[precip_col].min().reset_index()
print("Nan")
min_precip_df_by_month_df = min_precip_df_by_month_df.rename(columns={precip_col: "min_precip_month"})
print(min_precip_df_by_month_df.isna().sum())
min_precip_df_by_month_df

Nan
loc_group           0
climate_type        0
year                0
season              0
month               0
min_precip_month    0
dtype: int64


,loc_group,climate_type,year,season,month,min_precip_month
0,0,BSh,2014,3,9,12.95
1,0,BSh,2014,3,10,0.18
2,0,BSh,2014,3,11,0.63
3,0,BSh,2014,4,12,2.68
4,0,BSh,2015,1,3,9.29
...,...,...,...,...,...,...
13373,513,Dfb,2016,2,6,26.58
13374,513,Dfb,2016,2,7,43.82
13375,513,Dfb,2016,2,8,10.04
13376,513,Dfb,2016,4,1,1.88


In [23]:
min_wind_df_by_month_df = df.groupby(group_cols)[wind_col].min().reset_index()
print("Nan")
min_wind_df_by_month_df = min_wind_df_by_month_df.rename(columns={wind_col: "min_wind_month"})
print(min_wind_df_by_month_df.isna().sum())
min_wind_df_by_month_df

Nan
loc_group         0
climate_type      0
year              0
season            0
month             0
min_wind_month    0
dtype: int64


,loc_group,climate_type,year,season,month,min_wind_month
0,0,BSh,2014,3,9,5877.00
1,0,BSh,2014,3,10,5846.58
2,0,BSh,2014,3,11,5803.49
3,0,BSh,2014,4,12,5775.03
4,0,BSh,2015,1,3,5795.22
...,...,...,...,...,...,...
13373,513,Dfb,2016,2,6,5716.83
13374,513,Dfb,2016,2,7,5716.97
13375,513,Dfb,2016,2,8,5726.45
13376,513,Dfb,2016,4,1,5317.89


In [24]:
max_precip_df_by_month_df = df.groupby(group_cols)[precip_col].max().reset_index()
print("Nan")
max_precip_df_by_month_df = max_precip_df_by_month_df.rename(columns={precip_col: "max_precip_month"})
print(max_precip_df_by_month_df.isna().sum())
max_precip_df_by_month_df

Nan
loc_group           0
climate_type        0
year                0
season              0
month               0
max_precip_month    0
dtype: int64


,loc_group,climate_type,year,season,month,max_precip_month
0,0,BSh,2014,3,9,101.90
1,0,BSh,2014,3,10,62.95
2,0,BSh,2014,3,11,62.88
3,0,BSh,2014,4,12,51.45
4,0,BSh,2015,1,3,130.34
...,...,...,...,...,...,...
13373,513,Dfb,2016,2,6,81.43
13374,513,Dfb,2016,2,7,122.20
13375,513,Dfb,2016,2,8,57.45
13376,513,Dfb,2016,4,1,3.75


In [25]:
max_wind_df_by_month_df = df.groupby(group_cols)[wind_col].max().reset_index()
print("Nan")
max_wind_df_by_month_df = max_wind_df_by_month_df.rename(columns={wind_col: "max_wind_month"})
print(max_wind_df_by_month_df.isna().sum())
max_wind_df_by_month_df

Nan
loc_group         0
climate_type      0
year              0
season            0
month             0
max_wind_month    0
dtype: int64


,loc_group,climate_type,year,season,month,max_wind_month
0,0,BSh,2014,3,9,5903.36
1,0,BSh,2014,3,10,5880.70
2,0,BSh,2014,3,11,5859.14
3,0,BSh,2014,4,12,5847.35
4,0,BSh,2015,1,3,5857.99
...,...,...,...,...,...,...
13373,513,Dfb,2016,2,6,5782.79
13374,513,Dfb,2016,2,7,5817.80
13375,513,Dfb,2016,2,8,5784.95
13376,513,Dfb,2016,4,1,5420.93


In [26]:
mean_precip_df_by_month_df = df.groupby(group_cols)[precip_col].mean().reset_index()
print("Nan")
mean_precip_df_by_month_df = mean_precip_df_by_month_df.rename(columns={precip_col: "average_precip_month"})
print(mean_precip_df_by_month_df.isna().sum())
mean_precip_df_by_month_df

Nan
loc_group               0
climate_type            0
year                    0
season                  0
month                   0
average_precip_month    0
dtype: int64


,loc_group,climate_type,year,season,month,average_precip_month
0,0,BSh,2014,3,9,68.175667
1,0,BSh,2014,3,10,22.790000
2,0,BSh,2014,3,11,30.213333
3,0,BSh,2014,4,12,25.006129
4,0,BSh,2015,1,3,61.507742
...,...,...,...,...,...,...
13373,513,Dfb,2016,2,6,54.744333
13374,513,Dfb,2016,2,7,74.593226
13375,513,Dfb,2016,2,8,33.114194
13376,513,Dfb,2016,4,1,3.017097


In [27]:
mean_wind_df_by_month_df = df.groupby(group_cols)[wind_col].mean().reset_index()
print("Nan")
mean_wind_df_by_month_df = mean_wind_df_by_month_df.rename(columns={wind_col: "average_wind_month"})
print(mean_wind_df_by_month_df.isna().sum())
mean_wind_df_by_month_df

Nan
loc_group             0
climate_type          0
year                  0
season                0
month                 0
average_wind_month    0
dtype: int64


,loc_group,climate_type,year,season,month,average_wind_month
0,0,BSh,2014,3,9,5891.246000
1,0,BSh,2014,3,10,5870.056129
2,0,BSh,2014,3,11,5828.340333
3,0,BSh,2014,4,12,5806.259677
4,0,BSh,2015,1,3,5824.198710
...,...,...,...,...,...,...
13373,513,Dfb,2016,2,6,5756.482000
13374,513,Dfb,2016,2,7,5777.837097
13375,513,Dfb,2016,2,8,5762.954839
13376,513,Dfb,2016,4,1,5372.449355


In [28]:
min_precip_df_by_df = df.groupby(season_group_cols)[precip_col].min().reset_index()
print("Nan")
min_precip_df_by_df = min_precip_df_by_df.rename(columns={precip_col: "min_precip"})
print(min_precip_df_by_df.isna().sum())
min_precip_df_by_df

Nan
loc_group       0
climate_type    0
year            0
season          0
min_precip      0
dtype: int64


,loc_group,climate_type,year,season,min_precip
0,0,BSh,2014,3,0.18
1,0,BSh,2014,4,2.68
2,0,BSh,2015,1,0.22
3,0,BSh,2015,2,0.00
4,0,BSh,2015,3,1.27
...,...,...,...,...,...
5663,513,Dfb,2015,3,0.05
5664,513,Dfb,2015,4,0.07
5665,513,Dfb,2016,1,0.33
5666,513,Dfb,2016,2,10.04


In [29]:
min_wind_df_by_df = df.groupby(season_group_cols)[wind_col].min().reset_index()
print("Nan")
min_wind_df_by_df = min_wind_df_by_df.rename(columns={wind_col: "min_wind"})
print(min_wind_df_by_df.isna().sum())
min_wind_df_by_df

Nan
loc_group       0
climate_type    0
year            0
season          0
min_wind        0
dtype: int64


,loc_group,climate_type,year,season,min_wind
0,0,BSh,2014,3,5803.49
1,0,BSh,2014,4,5775.03
2,0,BSh,2015,1,5795.22
3,0,BSh,2015,2,5882.54
4,0,BSh,2015,3,5833.39
...,...,...,...,...,...
5663,513,Dfb,2015,3,5436.90
5664,513,Dfb,2015,4,5289.25
5665,513,Dfb,2016,1,5430.10
5666,513,Dfb,2016,2,5716.83


In [30]:
average_precip_df_by_df = df.groupby(season_group_cols)[precip_col].mean().reset_index()
print("Nan")
average_precip_df_by_df = average_precip_df_by_df.rename(columns={precip_col: "average_precip"})
print(average_precip_df_by_df.isna().sum())
average_precip_df_by_df

Nan
loc_group         0
climate_type      0
year              0
season            0
average_precip    0
dtype: int64


,loc_group,climate_type,year,season,average_precip
0,0,BSh,2014,3,40.199560
1,0,BSh,2014,4,25.006129
2,0,BSh,2015,1,59.785326
3,0,BSh,2015,2,8.925978
4,0,BSh,2015,3,20.642198
...,...,...,...,...,...
5663,513,Dfb,2015,3,18.884835
5664,513,Dfb,2015,4,5.623889
5665,513,Dfb,2016,1,26.424783
5666,513,Dfb,2016,2,54.144130


In [31]:
average_wind_df = df.groupby(season_group_cols)[wind_col].mean().reset_index()

print("Nan")
average_wind_df = average_wind_df.rename(columns={wind_col: "average_wind"})
print(average_wind_df.isna().sum())
average_wind_df

Nan
loc_group       0
climate_type    0
year            0
season          0
average_wind    0
dtype: int64


,loc_group,climate_type,year,season,average_wind
0,0,BSh,2014,3,5863.289341
1,0,BSh,2014,4,5806.259677
2,0,BSh,2015,1,5842.334239
3,0,BSh,2015,2,5909.821522
4,0,BSh,2015,3,5873.920330
...,...,...,...,...,...
5663,513,Dfb,2015,3,5602.831538
5664,513,Dfb,2015,4,5400.480556
5665,513,Dfb,2016,1,5575.357935
5666,513,Dfb,2016,2,5765.858804


In [32]:
max_precip_df_by_df = df.groupby(season_group_cols)[precip_col].max().reset_index()
print("Nan")
max_precip_df_by_df = max_precip_df_by_df.rename(columns={precip_col: "max_precip"})
print(max_precip_df_by_df.isna().sum())
max_precip_df_by_df

Nan
loc_group       0
climate_type    0
year            0
season          0
max_precip      0
dtype: int64


,loc_group,climate_type,year,season,max_precip
0,0,BSh,2014,3,101.90
1,0,BSh,2014,4,51.45
2,0,BSh,2015,1,130.34
3,0,BSh,2015,2,29.75
4,0,BSh,2015,3,51.04
...,...,...,...,...,...
5663,513,Dfb,2015,3,59.20
5664,513,Dfb,2015,4,22.60
5665,513,Dfb,2016,1,94.05
5666,513,Dfb,2016,2,122.20


In [33]:
max_precip_df_by_df = df.groupby(season_group_cols)[precip_col].max().reset_index()
print("Nan")
max_precip_df_by_df = max_precip_df_by_df.rename(columns={precip_col: "max_precip"})
print(max_precip_df_by_df.isna().sum())
max_precip_df_by_df

Nan
loc_group       0
climate_type    0
year            0
season          0
max_precip      0
dtype: int64


,loc_group,climate_type,year,season,max_precip
0,0,BSh,2014,3,101.90
1,0,BSh,2014,4,51.45
2,0,BSh,2015,1,130.34
3,0,BSh,2015,2,29.75
4,0,BSh,2015,3,51.04
...,...,...,...,...,...
5663,513,Dfb,2015,3,59.20
5664,513,Dfb,2015,4,22.60
5665,513,Dfb,2016,1,94.05
5666,513,Dfb,2016,2,122.20


In [34]:
max_wind_df_by_df = df.groupby(season_group_cols)[wind_col].max().reset_index()
print("Nan")
max_wind_df_by_df = max_wind_df_by_df.rename(columns={wind_col: "max_wind"})
print(max_wind_df_by_df.isna().sum())
max_wind_df_by_df

Nan
loc_group       0
climate_type    0
year            0
season          0
max_wind        0
dtype: int64


,loc_group,climate_type,year,season,max_wind
0,0,BSh,2014,3,5903.36
1,0,BSh,2014,4,5847.35
2,0,BSh,2015,1,5883.46
3,0,BSh,2015,2,5927.32
4,0,BSh,2015,3,5905.99
...,...,...,...,...,...
5663,513,Dfb,2015,3,5751.84
5664,513,Dfb,2015,4,5512.19
5665,513,Dfb,2016,1,5709.24
5666,513,Dfb,2016,2,5817.80


In [35]:
df = df.merge(max_wind_df_by_month_df, how='left', on=group_cols)
df['max_wind_month_diff'] = df[wind_col] - df['max_wind_month']
train_df['max_wind_month_diff'] = df[:len(train_df)]['max_wind_month_diff'].tolist()
test_df['max_wind_month_diff'] = df[len(train_df):]['max_wind_month_diff'].tolist()

In [36]:
df = df.merge(max_precip_df_by_month_df, how='left', on=group_cols)
df['max_precip_month_diff'] = df[precip_col] - df['max_precip_month']
train_df['max_precip_month_diff'] = df[:len(train_df)]['max_precip_month_diff'].tolist()
test_df['max_precip_month_diff'] = df[len(train_df):]['max_precip_month_diff'].tolist()

In [37]:
df = df.merge(max_precip_df_by_df, how='left', on=season_group_cols)
df['max_precip_diff'] = df[precip_col] - df['max_precip']
train_df['max_precip_diff'] = df[:len(train_df)]['max_precip_diff'].tolist()
test_df['max_precip_diff'] = df[len(train_df):]['max_precip_diff'].tolist()

In [38]:
df = df.merge(max_wind_df_by_df, how='left', on=season_group_cols)
df['max_wind_diff'] = df[wind_col] - df['max_wind']
train_df['max_wind_diff'] = df[:len(train_df)]['max_wind_diff'].tolist()
test_df['max_wind_diff'] = df[len(train_df):]['max_wind_diff'].tolist()

In [39]:
df = df.merge(min_precip_df_by_df, how='left', on=season_group_cols)
df['min_precip_diff'] = df[precip_col] - df['min_precip']
train_df['min_precip_diff'] = df[:len(train_df)]['min_precip_diff'].tolist()
test_df['min_precip_diff'] = df[len(train_df):]['min_precip_diff'].tolist()

In [40]:
df = df.merge(min_wind_df_by_df, how='left', on=season_group_cols)
df['min_wind_diff'] = df[wind_col] - df['min_wind']
train_df['min_wind_diff'] = df[:len(train_df)]['min_wind_diff'].tolist()
test_df['min_wind_diff'] = df[len(train_df):]['min_wind_diff'].tolist()

In [41]:
df = df.merge(average_precip_df_by_df, how='left', on=season_group_cols)
df['average_precip_diff'] = df[precip_col] - df['average_precip']
train_df['average_precip_diff'] = df[:len(train_df)]['average_precip_diff'].tolist()
test_df['average_precip_diff'] = df[len(train_df):]['average_precip_diff'].tolist()

In [42]:
df = df.merge(average_wind_df, how='left', on=season_group_cols)
df['average_wind_diff'] = df[wind_col] - df['average_wind']
train_df['average_wind_diff'] = df[:len(train_df)]['average_wind_diff'].tolist()
test_df['average_wind_diff'] = df[len(train_df):]['average_wind_diff'].tolist()

In [43]:
df = df.merge(mean_precip_df_by_month_df, how='left', on=group_cols)
df['average_precip_month_diff'] = df[precip_col] - df['average_precip_month']
train_df['average_precip_month_diff'] = df[:len(train_df)]['average_precip_month_diff'].tolist()
test_df['average_precip_month_diff'] = df[len(train_df):]['average_precip_month_diff'].tolist()

In [44]:
df = df.merge(mean_wind_df_by_month_df, how='left', on=group_cols)
df['average_wind_month_diff'] = df[wind_col] - df['average_wind_month']
train_df['average_wind_month_diff'] = df[:len(train_df)]['average_wind_month_diff'].tolist()
test_df['average_wind_month_diff'] = df[len(train_df):]['average_wind_month_diff'].tolist()

In [45]:
df = df.merge(min_precip_df_by_month_df, how='left', on=group_cols)
df['min_precip_month_diff'] = df[precip_col] - df['min_precip_month']
train_df['min_precip_month_diff'] = df[:len(train_df)]['min_precip_month_diff'].tolist()
test_df['min_precip_month_diff'] = df[len(train_df):]['min_precip_month_diff'].tolist()

In [46]:
df = df.merge(min_wind_df_by_month_df, how='left', on=group_cols)
df['min_wind_month_diff'] = df[wind_col] - df['min_wind_month']
train_df['min_wind_month_diff'] = df[:len(train_df)]['min_wind_month_diff'].tolist()
test_df['min_wind_month_diff'] = df[len(train_df):]['min_wind_month_diff'].tolist()

In [47]:
df['precip_range'] = df['max_precip'] - df['min_precip']
df['delta_precip'] = (df[precip_col] - df['min_precip']) / df['precip_range']
train_df['delta_precip'] = df[:len(train_df)]['delta_precip'].tolist()
test_df['delta_precip'] = df[len(train_df):]['delta_precip'].tolist()

In [48]:
df['wind_range'] = df['max_wind'] - df['min_wind']
df['delta_wind'] = (df[wind_col] - df['min_wind']) / df['wind_range']
train_df['delta_wind'] = df[:len(train_df)]['delta_wind'].tolist()
test_df['delta_wind'] = df[len(train_df):]['delta_wind'].tolist()

In [49]:
df['precip_range_month'] = df['max_precip_month'] - df['min_precip_month'] 
df['delta_precip_month'] = (df[precip_col] - df['min_precip_month']) / df['precip_range_month']
train_df['delta_precip_month'] = df[:len(train_df)]['delta_precip_month'].tolist()
test_df['delta_precip_month'] = df[len(train_df):]['delta_precip_month'].tolist()

In [50]:
df['wind_range_month'] = df['max_wind_month'] - df['min_wind_month'] 
df['delta_wind_month'] = (df[wind_col] - df['min_wind_month']) / df['wind_range_month']
train_df['delta_wind_month'] = df[:len(train_df)]['delta_wind_month'].tolist()
test_df['delta_wind_month'] = df[len(train_df):]['delta_wind_month'].tolist()

In [51]:
train_df['precip_range'] = df[:len(train_df)]['precip_range'].tolist()
test_df['precip_range'] = df[len(train_df):]['precip_range'].tolist()

train_df['precip_range_month'] = df[:len(train_df)]['precip_range_month'].tolist()
test_df['precip_range_month'] = df[len(train_df):]['precip_range_month'].tolist()


In [52]:
train_df['wind_range'] = df[:len(train_df)]['wind_range'].tolist()
test_df['wind_range'] = df[len(train_df):]['wind_range'].tolist()

train_df['wind_range_month'] = df[:len(train_df)]['wind_range_month'].tolist()
test_df['wind_range_month'] = df[len(train_df):]['wind_range_month'].tolist()

In [53]:
best_cols

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__c

In [54]:
features = best_cols[:-4] + [
    'min_precip_month_diff', 
    'average_precip_month_diff', 
    'min_precip_diff', 
    'average_precip_diff',
#     'delta_precip_month',
#     'delta_precip',
    'precip_range',
    'precip_range_month',  
    'min_wind_month_diff', 
    'average_wind_month_diff', 
    'min_wind_diff', 
    'average_wind_diff',
#     'delta_wind_month',
#     'delta_wind',
    'wind_range',
    'wind_range_month'
]

In [55]:
for c in features:
    if test_df[c].isnull().sum() > 0:
        print(c, train_df[c].isnull().sum())

In [56]:
# X = train_df[features].values
# y = train_df[target[0]].values

# skf = KFold(n_splits=5, random_state=2048, shuffle=True)
# skf.get_n_splits(X, y)

# train_index_list = []
# test_index_list = []
# train_x_list = []
# val_x_list = []
# train_y_list = []
# val_y_list = []

# for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
#     train_index_list.append(train_index)
#     test_index_list.append(test_index)

#     train_x_list.append(X[train_index, :])
#     val_x_list.append(X[test_index, :])
#     train_y_list.append(y[train_index])
#     val_y_list.append(y[test_index])    
# def train_catboost(x, y):
#     print(x.shape, y.shape)
    
#     cat_models = []
#     for i in tqdm(range(5)):
#         train_x, val_x = train_x_list[i], val_x_list[i]
#         train_y, val_y = train_y_list[i], val_y_list[i]
#         clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
#                            devices='0:1')
#         clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
#         cat_models.append(clf)
    
#     return cat_models
# cat_models = train_catboost(X, y)

# preds = []
# for cat_model in tqdm(cat_models):
#     preds.append(cat_model.predict(test_df[features].values))
# s = 0
# for pred in preds:
#     s += pred
# s = s/5
# result_df = pd.read_csv('./sample_solution.csv')
# result_df['contest-tmp2m-14d__tmp2m'] = s
# result_df.to_csv('submission_precip_fe.csv', index=False)

# best_df = pd.read_csv('best_sub.csv')
# y_best = best_df[target[0]].values

# from numpy import dot
# from numpy.linalg import norm

# cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
# print("cos_sim with best submission:", cos_sim)

In [57]:
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train(x, y):
    print(x.shape, y.shape)
    
    models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = AdaBoostRegressor(
            DecisionTreeRegressor(max_depth=30), n_estimators=300, random_state=np.random.RandomState(1))
        clf.fit(train_x, train_y)
        print("Val score:", mean_squared_error(clf.predict(val_x), val_y, squared=False))
        models.append(clf)
    
    return models
models = train(X, y)

preds = []
for model in tqdm(models):
    preds.append(model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
result_df = pd.read_csv('./sample_solution.csv')
result_df['contest-tmp2m-14d__tmp2m'] = s
result_df.to_csv('submission_rf.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:00,  7.27it/s]


(375734, 71) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

In [ ]:
preds = []
for model in tqdm(models):
    preds.append(model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values
result_df = pd.read_csv('./sample_solution.csv')
t = s * 0.1 + y_best * 0.9
result_df['contest-tmp2m-14d__tmp2m'] = t
result_df.to_csv('submission_rf.csv', index=False)


from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, t)/(norm(y_best)*norm(t))
print("cos_sim with best submission:", cos_sim)